In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
from functools import partial
from matplotlib import pyplot as plt
from scipy.stats import percentileofscore
from collections import defaultdict
sns.set(rc={'figure.facecolor':'white'})

## Data loading + munging 

In [34]:
df = pd.read_csv("../data/drugs_agency_2019_20210308.csv")

### Inspect Data Columns

In [35]:
df.columns

Index(['dm_offender_race_ethnicity', 'dm_offender_sex', 'dm_offender_age',
       'offender_seq_num', 'offense_type_id', 'location_id',
       'cleared_except_id', 'arrest_type_id', 'criminal_act_id',
       'relationship_id', 'prop_loss_id', 'prop_desc_id', 'property_value',
       'suspected_drug_type_id', 'est_drug_qty', 'drug_measure_type_id',
       'state_abbr', 'agency'],
      dtype='object')

### Location Mapping

In [36]:
location_dict = {
    13: "Street",
    20: "Home",
    18: "Street",
    14: "Hotel/Motel",
    8: "Store",
    7: "Store",
    23: "Store",
    41: "School-Elementary",
    12: "Store",
}
location_dict = defaultdict(lambda: 'Other', location_dict)
df["location"] = df.location_id.map(location_dict)

### Property Value Mapping

In [37]:
def property_value_func(n):
    if n == 0:
        return "n==0"
    if n > 0 and n <= 10:
        return "0 < n <= 10"
    if n > 10 and n <= 100:
        return "10 < n <= 100"
    if n > 100 and n <= 1000:
        return "100 < n <= 1000"
    if n > 1000:
        return "n > 1000"
    return "Unknown"

#df["property_value"] = df.property_value.apply(property_value_func)

### Property Description Mapping

In [38]:
property_description_dict = {
    10: "Drugs",
    11: "Drug Equipment",
    20: "Money",
    13: "Firearms",
    3: "Automobile",
}
property_description_dict = defaultdict(lambda: 'Other', property_description_dict)
df["property_description"] = df.prop_desc_id.map(property_description_dict.get)

In [42]:
df[df.property_description == "Drug Equipment"]["suspected_drug_type_id"].value_counts()

5.0     363601
12.0    267831
4.0      80174
16.0     53643
8.0      43063
17.0     30026
2.0      29479
1.0      26473
18.0     16329
15.0     10756
11.0      9242
3.0       6174
13.0      5713
7.0       3289
9.0       2708
14.0      2064
6.0       1512
10.0       476
Name: suspected_drug_type_id, dtype: int64

### Property Loss Mapping

In [7]:
property_loss_dict = {
    6: "Seized",
    7: "Stolen",
    5: "Recovered",
}
property_loss_dict = defaultdict(lambda: 'Other', property_loss_dict)
df["property_loss"] = df.prop_loss_id.map(property_loss_dict)

In [8]:
def age_func(x):
    if x < 18:
        return "[0, 18)"
    if x < 21:
        return "[18, 21)"
    if x < 24:
        return "[21, 24)"
    if x < 31:
        return "[24, 31)"
    if x < 42:
        return "[31, 42)"
    if x < 120:
        return "[42, 120)"
    return "Unknown"
df["dm_offender_age"] = df.dm_offender_age.apply(age_func)

In [9]:
criminal_act_dict = {
    6: "Possessing",
    3: "Distributing",
    8: "Consuming",
    1: "Buying",
    7: "Transporting",
    2: "Cultivating/Manufacturing/Other",
}
criminal_act_dict = defaultdict(lambda: 'Other', criminal_act_dict)
df["criminal_act"] = df.criminal_act_id.map(criminal_act_dict.get)

### Replace NaN drug quantities with 0, and drug measure type to unknown

In [10]:
df.fillna({"est_drug_qty": -1}, inplace=True)
df['drug_measure_type_id'] = df['drug_measure_type_id'].replace(np.nan, 11)

no_quantity = (df["est_drug_qty"] == -1) | ((df["drug_measure_type_id"] == 11) & (df["est_drug_qty"] != 0))

no_quantity_df = df[no_quantity]
no_quantity_df["drug_quantity_quantile"] = "Unknown"

df = df[~no_quantity]

/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


### Convert drug quantity to uniform measures

In [11]:
quantity_dict = {
    1: 1,
    2: 1000,
    3: 28.3495,
    4: 453.592,
    5: 1,
    6: 1000,
    7: 28.4131,
    8: 3785.41,
    9: 1,
    10: 1,
    11: 0
}
quantity_conv = lambda x: quantity_dict[x["drug_measure_type_id"]] * x["est_drug_qty"]

### Convert numerical columns to strings

In [12]:
unit_type = [
    "Gram",
    "Kilogram",
    "Ounce",
    "Pound",
    "Milliliter",
    "Liter",
    "Fluid Ounce",
    "Gallon",
    "Dosage Unit",
    "Number of Plants",
    "Not Reported",
]
unit_type_dict = {i + 1: u for i, u in enumerate(unit_type)}
unit_type_dict = defaultdict(lambda: 'Unknown', unit_type_dict)

In [13]:
drug_state_dict = {
    "Gram": "Mass",
    "Kilogram": "Mass",
    "Ounce": "Mass",
    "Pound": "Mass",
    "Milliliter": "Volume",
    "Liter": "Volume",
    "Fluid Ounce": "Volume",
    "Gallon": "Volume",
    "Dosage Unit": "Unit",
    "Number of Plants": "Unit",
}
drug_state_dict = defaultdict(lambda: 'Unknown', drug_state_dict)

In [14]:
df["est_drug_qty_standard"] = df.apply(quantity_conv, axis=1)
df.drug_measure_type_id = df.drug_measure_type_id.map(unit_type_dict)
df["drug_state"] = df.drug_measure_type_id.map(drug_state_dict)

### Quantify by Quartiles

In [15]:
quantile_func = partial(np.quantile, q=[0.2, 0.4, 0.6, 0.8, 1])
quantiles = df.groupby(["suspected_drug_type_id", "drug_state"], dropna=False)["est_drug_qty_standard"].apply(quantile_func)
quantile_conv = lambda x: str(percentileofscore(quantiles[(x["suspected_drug_type_id"], x["drug_state"])], x["est_drug_qty"], kind="strict"))
df["drug_quantity_quantile"] = df.apply(quantile_conv, axis=1)
#df = df.append(no_quantity_df)

In [16]:
drug_types = [
    "Crack Cocaine",
    "Cocaine",
    "Other",
    "Heroin",
    "Marijuana",
    "Other",
    "Other",
    "Other",
    "Other",
    "Other",
    "Other",
    "Meth/Amphetamines",
    "Other",
    "Other",
    "Other",
    "Other",
    "Other",
    "Other",
]
drug_type_dict = {i + 1: d for i, d in enumerate(drug_types)}
df["suspected_drug_type"] = df.suspected_drug_type_id.map(drug_type_dict)

### Cleanup

In [17]:
df.loc[(df.dm_offender_sex == " "), "dm_offender_sex"] = "U"
df.fillna({"property_value": "Unknown"}, inplace=True)
df.fillna({"suspected_drug_type": "Unknown"}, inplace=True)
df.fillna({"property_description": "Unknown"}, inplace=True)
df.fillna({"criminal_act": "Cultivating/Manufacturing/Other"}, inplace=True)

In [18]:
offense_type_dict = {
    16: "Drug/Narcotics Violations",
    35: "Drug Equipment Violations", 
}
offense_type_dict = defaultdict(lambda: 'Unknown', offense_type_dict)

In [19]:
arrest_type_dict = {
    1: "On View",
    2: "Summoned/Cited",
    3: "Taken into Custody",
}
arrest_type_dict = defaultdict(lambda: 'Not Arrested', arrest_type_dict)

In [20]:
df["offense_type"] = df.offense_type_id.map(offense_type_dict.get)
df["arrest_type"] = df.arrest_type_id.map(arrest_type_dict.get)

In [21]:
df.fillna({"arrest_type": "Not Arrested"}, inplace=True)

In [28]:
to_drop = set(["location_id", "agency", "property_loss", "property_value", "prop_loss_id", "suspected_drug_type_id", "offense_type_id", "arrest_type_id", "cleared_except_id", "drug_state", "prop_desc_id", "criminal_act_id", "est_drug_qty", "est_drug_qty_standard", "drug_measure_type_id", "state_abbr", "offender_seq_num", "relationship_id"]) & set(df.columns)
df = df.drop(to_drop, axis=1)

In [29]:
df.nunique()

dm_offender_race_ethnicity    5
dm_offender_sex               3
dm_offender_age               7
location                      6
property_description          6
criminal_act                  6
drug_quantity_quantile        5
suspected_drug_type           6
offense_type                  2
arrest_type                   4
dtype: int64

In [30]:
df.to_csv("../data/NIBRS_drug_20210308.csv", index=False)